In [1]:
!pip install transformers datasets sentencepiece accelerate -q



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="data.csv")
dataset


/Users/nikhilkumar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 96
    })
})

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
def preprocess(batch):
    inputs = ["fix: " + text for text in batch['input']]
    outputs = batch['target']

    model_inputs = tokenizer(
        inputs, max_length=128, truncation=True, padding="max_length"
    )

    labels = tokenizer(
        outputs, max_length=128, truncation=True, padding="max_length"
    ).input_ids

    model_inputs["labels"] = labels
    return model_inputs

dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/96 [00:00<?, ? examples/s]

In [5]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./grammar-model",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    learning_rate=3e-5,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    do_eval=False,
    report_to="none"      # 🚀 turns off wandb forever
)




In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"]
)

trainer.train()



In [ ]:
from google.colab import files
!zip -r grammar-model.zip grammar-model
files.download("grammar-model.zip")


In [ ]:
import os
os.environ.pop("WANDB_DISABLED", None)


In [ ]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="./grammar-model", tokenizer="./grammar-model")

test_sentences = [
    "i am go to school",
    "he dont like it",
    "hw r u",
    "sry for late reply",
]

for s in test_sentences:
    print("INPUT:", s)
    print("OUTPUT:", pipe(s)[0]["generated_text"])
    print()


In [ ]:
pipe = pipeline(
    "text2text-generation",
    model="grammar-model/checkpoint-72",
    tokenizer="grammar-model/checkpoint-72"
)


In [ ]:
print(pipe("i am go school")[0]["generated_text"])


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline

tokenizer = T5Tokenizer.from_pretrained("grammar-model/checkpoint-72", use_fast=False)
model = T5ForConditionalGeneration.from_pretrained("grammar-model/checkpoint-72")

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer
)

print(pipe("fix: i am go school")[0]["generated_text"])
print(pipe("fix: he dont like it")[0]["generated_text"])
print(pipe("fix: hw r u")[0]["generated_text"])


In [ ]:
!zip -r grammar-model.zip grammar-model/checkpoint-72


In [ ]:
from google.colab import files
files.download("grammar-model.zip")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!zip -r /content/drive/MyDrive/grammar-model.zip grammar-model/checkpoint-72


In [ ]:
tokenizer.save_pretrained("final-model")


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load your checkpoint
model = T5ForConditionalGeneration.from_pretrained("grammar-model/checkpoint-72")
tokenizer = T5Tokenizer.from_pretrained("t5-base")   # IMPORTANT

# Save full model
model.save_pretrained("final-model")
tokenizer.save_pretrained("final-model")
